<a href="https://colab.research.google.com/github/kake01/Bot/blob/master/musuka.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#ドライブをマウントする
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# テキストデータの分割
import sys
import os
import chardet
import tensorflow as tf
import os.path

from os.path import join, splitext, split

# 入力ディレクトリと出力ディレクトリを指定
readdir = sys.argv[1]
outdir ="drive/My Drive/Bot/Serif"

# 天空の城ラピュタのテキストデータを代入
file_data = open('drive/My Drive/Bot/TextData.txt', 'r',encoding='utf-8')
lines = file_data.readlines()

#読み込んだテキストファイルを１行ずつ表示
for line in lines:

# キャラ名を取得
    char_name = line[:line.find(u"「")]
    outfname = os.path.join(outdir, char_name + ".txt")
#キャラ名のファイルがあるかどうか確認
    if os.path.exists(outfname):
        # ある場合は上書きモードで
        outfp = open(outfname, "a")
    else:
        # ない場合は新規作成
        outfp = open(outfname, "w")
        
    # セリフのみ抽出
    line_format = line[line.find(u"「") + 1:line.find(u"」")] + "\n"
    # セリフ書き込み
    outfp.write(line_format)


In [0]:
# 蛇の目インストール
pip install janome

In [0]:
from janome.tokenizer import Tokenizer
t = Tokenizer()
for token in t.tokenize(u'パズー君。君を誤解していた。許してくれたまえ。君がこの方を海賊から守るために、奮戦してくれたとは知らなかったんだ。'):
  print(token)

# text = 'パズー君。君を誤解していた。許してくれたまえ。君がこの方を海賊から守るために、奮戦してくれたとは知らなかったんだ。'
# t.tokenize(text, wakati= True)

In [0]:
from collections import Counter

BEGIN = '__BEGIN__'
END = '__END__'

""" 文章を3単語の組にして返す"""
def get_three_words_list(sentence):
  t = Tokenizer()
  words = t.tokenize(sentence, wakati = True)
  words = [BEGIN] + words + [END]
  three_words_list = []
  for i in range(len(words) - 2):
    three_words_list.append(tuple(words[i:i+3]))
  return three_words_list


sentences = ['おいしいビールを飲もう', 'ビールを飲もう','おいしいビールは生']
print(sentences)
three_words_list = []
for sentence in sentences:
  three_words_list += get_three_words_list(sentence)
three_words_conut = Counter(three_words_list)
three_words_conut




['おいしいビールを飲もう', 'ビールを飲もう', 'おいしいビールは生']


Counter({('__BEGIN__', 'おいしい', 'ビール'): 2,
         ('__BEGIN__', 'ビール', 'を'): 1,
         ('おいしい', 'ビール', 'は'): 1,
         ('おいしい', 'ビール', 'を'): 1,
         ('は', '生', '__END__'): 1,
         ('を', '飲も', 'う'): 2,
         ('ビール', 'は', '生'): 1,
         ('ビール', 'を', '飲も'): 2,
         ('飲も', 'う', '__END__'): 2})

In [57]:
""" マルコス連鎖での文章生成用の辞書データを生成する """
def generate_markov_dict(three_words_conut):
  markov_dict = {}
  for three_words, count in three_words_conut.items():
    two_words = three_words[:2]
    next_word = three_words[2]
    if two_words not in markov_dict:
      markov_dict[two_words] = {'words': [], 'weights': []}
    markov_dict[two_words]['words'].append(next_word)
    markov_dict[two_words]['weights'].append(count)
  
  return markov_dict

markov_dict = generate_markov_dict(three_words_conut)
markov_dict

defaultdict(int, {'おいしい': 2, 'ビール': 1})

In [56]:
from collections import defaultdict

def get_first_word_and_count(three_words_conut):
  """ 最初の単語を選択するための辞書データを作成する """
  first_word_count = defaultdict(int)

  for three_words, count in three_words_conut.items():
    if three_words[0] == BEGIN:
      next_word = three_words[1]
      first_word_count[next_word] += count
  return first_word_count

get_first_word_and_count(three_words_conut)

defaultdict(int, {'おいしい': 2, 'ビール': 1})

In [62]:
# p151
def get_first_word_weights(three_words_conut):
  """ 最初の単語と重みのリストを作成する　"""
  first_word_count = get_first_word_and_count(three_words_conut)
  words = []
  weights = []
  for word, count in first_word_count.items():
    words.append(word)
    weights.append(count)

  return words, weights


first_words, first_weights = get_first_word_weights(three_words_conut)
first_words, first_weights

(['おいしい', 'ビール'], [2, 1])

In [0]:
import random

def generate_text(first_words, first_weights, markov_dict):
  """ 入力された辞書データを元に文章を生成する """
  first_word = random.choices(first_words, weights = first_weights)[0]
  generate_words = [BEGIN, first_word]
  while True:
    pair = tuple(generate_words[-2:])
    words = markov_dict[pair]['words']
    weights = markov_dict[pair]['weights']
    next_word = random.choices(words, weights=weights)[0]
    if next_word == END:
      break
    generate_words.append(next_word)

  return ''.join(generate_words[1:])

In [0]:
for _ in range(5):
  text = generate_text(first_words, first_weights, markov_dict)
  print(text)

In [0]:
import time
from tqdm import tqdm

three_words_list = []
# p159から再開

# テキストデータをもう少し前処理しないとダメかも?
